# Assignment part 3

By: Markus Trasberg 5105013, Melle Schoenmaker 5062306

In [ ]:
import numpy as np
import random as rand
from matplotlib import pyplot as plt
from math import *
from scipy.stats import nbinom
import pandas as pd


## Binomial Generator
A sample from a binomial distribution with specified amount of shots and probability of succes.
It works by iterating n times and each time generating a uniform random value between 0 and 1, if this value is less than the probability then the shot was a success and x is incremented by 1. 
Once the amount of iterations (k) reaches n, the number of successes (x) is returned

In [ ]:
# Binomial distribution

def binomgen(n, p):
    x = 0
    k = 0
    while k < n:
        u = rand.uniform(0, 1)
        if (u <= p):
            x += 1
        k += 1
    return x

## Negative Binomial Generator
A sample from a negative binomial distribution with specified amount of failures and probability of succes.
It works by iterating until the amount of failures (fail) reaches r.
Each iteration a uniform random variable between 0 and 1 is generated, if this value is higher than the probability then the shot was a failure and fail is incremented by 1.
Once the amount of failures (fail) reaches r, the number of iterations minus failures (k - fail) is returned

In [ ]:
# Negative Binomial Distribution

def nbinomgen(r, p):
    fail = 0
    k = 0
    while fail < r:
        u = rand.uniform(0, 1)
        if (u > p):
            fail += 1
        k += 1
    return k - fail

In [ ]:
parameters = [(10, 0.5), (10, 0.7), (50, 0.5)]
coinflip_params = [(1000, 0.5)]

To show that these distributions are close to the actual underlying distributions we generate the probability density functions of the distributions.
Comparing the two plots for each distribution with parameters (n, p) or (r, p) will show that they have a similar shape and thus the samples have similar frequencies. 

In [ ]:
for (n, p) in parameters:
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figwidth(20)
    values = [binomgen(n, p) for i in range(5000)]
    labels, counts = np.unique(values, return_counts=True)
    ax1.bar(labels, counts, align='center')
    ax1.set_xlim([-0.5, n + 0.5])
    ax1.set_xticks(np.arange(0, n+1, n // 10))
    ax1.set_title("Binomial Sample") 
    ax1.set_xlabel("Number of successes") 
    ax1.set_ylabel("Number of samples") 

    values = list(range(n+1))
    dist = [comb(n, i) * p ** i * (1-p) ** (n - i) for i in range(n + 1)]

    ax2.set_title("Binomial distribution probability density function") 
    ax2.set_xlabel("Number of samples") 
    ax2.set_ylabel("Probability") 
    
    ax2.set_xlim([-0.5, n + 0.5])
    ax2.set_xticks(np.arange(0, n+1, n // 10))
    ax2.bar(values, dist)

    

In [ ]:
for (r, p) in parameters:
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_figwidth(20)

    values = [nbinomgen(r, p) for i in range(5000)]
    labels, counts = np.unique(values, return_counts=True)
    ax1.bar(labels, counts, align='center')
    ax1.set_xlim([-0.5, max(values) + 0.5])
    ax1.set_xticks(np.arange(0, max(values)+1, max(values) // 10))
    ax1.set_title("Negative Binomial Distribution Sample")
    ax1.set_xlabel(f"Number of successes until {r} failures")
    ax1.set_ylabel("Number of samples")

    x = np.arange(0, max(values))
    ax2.bar(x, nbinom.pmf(x, r, 1 - p))
    ax2.set_title("Negative Binomial distribution probability density function")
    ax2.set_xlabel(f"Number of successes until {r} failures")
    ax2.set_ylabel("Probability")
    ax2.set_xticks(np.arange(0, max(x), max(x) // 10))

## Flipping a coin
For the game of flipping a coin, we use a similar strategy as before to get a random sample, however instead of only adding 1 to the counter for a success, we also substract 1 from the counter in case of a failure.
This way the counter keeps track of Sn throughout the N (1000) samples and the value of Sn divided by sqrt(N) is stored in an array for plotting.

As can be seen by rerunning the code below, the trajectory fluctuates around 0 with the eventual peak or dip due to randomness in the sampling method.

In [ ]:
for (N, p) in coinflip_params:
    Sn = 0
    n = 0
    values = [None] * (N+1)
    while n <= N:
        values[n] = Sn / sqrt(N)
        u = rand.uniform(0, 1)
        if (u <= p):
            Sn += 1
        else:
            Sn -= 1
        n += 1
    labels = [i for i in range(N+1)]
    fig, ax = plt.subplots(1, 1)
    fig.set_figwidth(20)
    ax.set_xlim([0, N])
    ax.set_xticks(np.arange(0, N, N // 10))
    ax.set_title("Sample gain")
    ax.set_xlabel(f"Number of trials")
    ax.set_ylabel("Gain (Sn / sqrt(N))")
    ax.plot(labels, values)

## Symmetry property

### Bernoulli scheme generator
To implement a Bernoulli scheme generator we take a similar approach as the one taken in ex1.
Given a probability p as an argument, the function generates a value uniformally at random and compares this to p.
If the random value was less than the value of the probability then the generator returns one
Otherwise the generator returns 0

### Cantor distribution generator
To implement a Cantor generator with a given probability p and a level of precision n we use the bernoulli generator defined above.
To start the generator initializes a result value as 0
Then iteratively, 2 * (1/3)**(i + 1) * bernoulli(p) is repeatedly added to the result which corresponds to generating the lower limits of the boundaries in the Cantor set C_i

In [ ]:
# Exercise 2 - Symmetry property
# Bernoulli Scheme Generator
def bernoulli(p):
    if (rand.uniform(0, 1) < p):
        return 1
    return 0

# Cantor generator

def cantor(n, p):
    res = 0
    for i in range(n):
        res += bernoulli(p) * 2 * 1/3**(i + 1)

    return res

def prob(x, labels, counts, N):
    counter = 0
    probability = 0
    for i in labels:
        if (x - 0.000001 <= i and x + 0.000001 >= i):
            return np.NaN
        if (x < i):
            break
        counter += 1
    for i in range(counter):
        probability += counts[i] / N
    return probability


### Cantor distribution sample
Below image shows a sample taken from the cantor distribution generator. It shows a uniform distribution between all of the potential values, these potential values are limited to the lower bounds of the intervals of a cantor set with precision level N

In [ ]:
fig, ax1 = plt.subplots(1, 1)
fig.set_figwidth(20)
N = 6
p = 0.5
binwidth = 3**-N
values = [cantor(N, p) for i in range(10000)]
labels, counts = np.unique(values, return_counts=True)
ax1.hist(values, bins=np.arange(0 - binwidth/2, 1 + binwidth/2, binwidth))
ax1.set_xlim([-binwidth/2, 1 + binwidth/2])
ax1.set_xticks(np.arange(0, 1, 3**-3))
ax1.set_title("Cantor Distribution Sample")
ax1.set_xlabel("Sample value")
ax1.set_ylabel("Number of samples")
plt.show()

### Symmetry Properties
#### With respect to 1/2
To verify the symmetry property with respect to 1/2 where X is equally distributed with 1-X, we plotted two distribution functions using the same sample set X that was generated using the Cantor generator. For the second plot, we inverted the values around 1/2. As our test confirms, the shape of the two plots is identical, thus confirming the reflection symmetry property.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
fig.set_figwidth(40)
fig.set_figheight(10)
N = 6
p = 0.5
binwidth = 3**-N

values1 = [cantor(N, p) for i in range(10000)]
x = np.arange(0, 1, binwidth)
labels1, counts1 = np.unique(values1, return_counts=True)
ax1.set_xlim([-binwidth/2, 1 + binwidth/2])
ax1.set_xticks(np.arange(0, 1, 3**-3))
ax1.set_title("Cantor Distribution Function")
ax1.set_xlabel("Value")
ax1.set_ylabel("Probability X<Value")
ax1.plot(x, [prob(i, labels1, counts1, np.sum(counts1)) for i in x])

ax2.hist(values1, bins=np.arange(0 - binwidth/2, 1 + binwidth/2, binwidth))
ax2.set_xlim([-binwidth/2, 1 + binwidth/2])
ax2.set_xticks(np.arange(0, 1, 3**-3))
ax2.set_title("Cantor Distribution Sample")
ax2.set_xlabel("Sample value")
ax2.set_ylabel("Number of samples")

values2 = [1 - i for i in values1]
labels2, counts2 = np.unique(values2, return_counts=True)
ax3.set_xlim([-binwidth/2, 1 + binwidth/2])
ax3.set_xticks(np.arange(0, 1, 3**-3))
ax3.set_title("Cantor Distribution Function")
ax3.set_xlabel("(1-X)")
ax3.set_ylabel("Probability (1-X)<Value")
ax3.plot(x, [prob(i, labels2, counts2, np.sum(counts1)) for i in x])

ax4.hist(values2, bins=np.arange(0 - binwidth/2, 1 + binwidth/2, binwidth))
ax4.set_xlim([-binwidth/2, 1 + binwidth/2])
ax4.set_xticks(np.arange(0, 1, 3**-3))
ax4.set_title("Cantor Distribution Sample")
ax4.set_xlabel("1 - Sample value")
ax4.set_ylabel("Number of samples")
plt.show()

#### Self-similarity
To verify the self-similarity property between the conditional distribution Y under the condition that Y belongs to the interval [0, 1/3] and the distribution Y/3, we plotted two distribution functions using the same sample set X that was generated using the Cantor generator. For the first plot, we exclude the labels and counts for values higher than 1/3. For the second plot, we divided the values by 3. As our test confirms, the shape of the two plots is identical, thus confirming the self-similarity property.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
fig.set_figwidth(40)
fig.set_figheight(10)
N = 6
p = 0.5
binwidth = 3**-N

values1 = [cantor(N, p) for i in range(10000)]

x = np.arange(0, 1/3, binwidth)
labels1, counts1 = np.unique(values1, return_counts=True)
count = 0
for i in labels1:
    if (i > 1/3):
        break
    count += 1
labels1 = labels1[0:count]
counts1 = counts1[0:count]

    
ax1.set_xlim(0, 1/3)
ax1.set_xticks(np.arange(0, 1/3, 3**-3))
ax1.set_title("Cantor Distribution Function")
ax1.set_xlabel("Y in the interval [0, 1/3]")
ax1.set_ylabel("Probability Y < Value on interval [0, 1/3]")
ax1.plot(x, [prob(i, labels1, counts1, sum(counts1)) for i in x])

ax2.hist(values1, bins=np.arange(0 - binwidth/2, 1 + binwidth/2, binwidth))
ax2.set_xlim([-binwidth/2, 1/3 + binwidth/2])
ax2.set_xticks(np.arange(0, 1/3, 3**-3))
ax2.set_title("Cantor Distribution Sample")
ax2.set_xlabel("Y in the interval [0, 1/3]")
ax2.set_ylabel("Number of samples")

binwidth /= 3

values2 = [i/3 for i in values1]
labels2, counts2 = np.unique(values2, return_counts=True)
ax3.set_xlim([-binwidth/2, 1/3 + binwidth/2])
ax3.set_xticks(np.arange(0, 1/3, 3**-3))
ax3.set_title("Cantor Distribution Function")
ax3.set_xlabel("Y / 3")
ax3.set_ylabel("Probability (Y / 3) < Value")
ax3.plot(x, [prob(i, labels2, counts2, sum(counts2)) for i in x])

ax4.hist(values2, bins=np.arange(0 - binwidth/2, 1 + binwidth/2, binwidth))
ax4.set_xlim([-binwidth/2, 1/3 + binwidth/2])
ax4.set_xticks(np.arange(0, 1/3, 3**-3))
ax4.set_title("Cantor Distribution Sample")
ax4.set_xlabel("Sample value / 3")
ax4.set_ylabel("Number of samples")
plt.show()

### Checking the distribution function
As shown for some of the previous questions we can also generate a distribution function of the sample using the probability function defined above.
To demonstrate we take a sample from the cantor generator, and create a range from 0 to 1 with steps of size 3**-N. For each of the values in this range we can calculate the probability that a value from the sample is below that value.

In [ ]:
fig, ax1 = plt.subplots(1, 1)
fig.set_figwidth(20)
N = 6
p = 0.5
binwidth = 3**-N
values = [cantor(N, p) for i in range(1000)]
x = np.arange(0, 1, binwidth)
labels, counts = np.unique(values, return_counts=True)
ax1.set_xlim([-binwidth/2, 1 + binwidth/2])
ax1.set_xticks(np.arange(0, 1, 3**-3))
ax1.set_title("Cantor Distribution Function")
ax1.set_xlabel("Value")
ax1.set_ylabel("Probability X<Value")
ax1.plot(x, [prob(i, labels, counts, sum(counts)) for i in x])

## Law of Large Numbers

The Law of Large Numbers says that as the number of randomly generated & identically distributed variables grows, the sample mean approaches the true mean. To test this out, we created a function graph_LLN which takes two parameters: a (mean) and b (standard deviation). The function loops through n values, that is from 1 to 10000, drawing a sample using normal distribution for every n value. We calculate the mean for each of those samples and plot them next to each other.





In [ ]:
def graph_LLN(a, b):
    result = []
    for n in range(1,10000):
        mean = np.mean(np.random.normal(a, b, n))
        result.append(mean)

    df = pd.DataFrame({ 'tries' : result})

    df.plot(legend=None)
    plt.title('Law of Large Numbers')

    plt.xlabel("Sample size")
    plt.ylabel("Sample mean averages")
    plt.show()

graph_LLN(0, 1)
graph_LLN(20, 50)

Let's test this out using a = 0 and b = 1. We notice that for small sample sizes, the means are scattered all over y-axis but as the sample size grows, the values start narrowing towards the true mean value, that is 0 in our example.

When testing this out with a = 20 and b = 50, we notice the converging takes significantly longer which is caused by the large standard deviation. Nonetheless, as sample size grows, it eventually narrows close to the true mean of 20.


## Central Limit Theorem
Central Limit Theorem states that the distribution of a sum of many independent random variables, each with its own known distribution, will be approximately normal. Let's verify this by testing on different sample sizes - 1, 10, 50 and 500. We have a function graph_CLT which takes as parameters a (mean) and b (standard deviation). Then, a sample is drawn n-times from normal distribution with the given parameters. For each sample, we calculate it's mean, substract a and multiply this by sqrt(n). This process is repeated N times (in our case N = 40) and all values are then plotted on the graph according to the sample sizes.

When plugging in a = 0 and b = 1, we clearly see that as n grows, the distribution of a sample variable starts shaping like a normal distribution (the bell curve).


In [ ]:
def graph_CLT(a,b):
    n_list = [1, 10, 50, 500]
    N = 40
    sample = []
    k = 0

    for i in range(len(n_list)):
        sample.append(list())

    # Loop through the n_list
    for n in n_list:
        for i in range(N):
            for j in range(n):
                s = np.mean(np.random.normal(a, b, n))
                z = np.sqrt(n) * (s - a)
                sample[k].append(z)
        k += 1


    k = 0
    # Generate graphs for every n in n_list
    fig, ax = plt.subplots(2, 2, figsize=(10, 8))
    for i in range(2):
        for j in range(2):
            ax[i, j].hist(sample[k], density = True, bins=75)
            ax[i, j].set_title(label = "N = " + str(n_list[k]))
            k += 1
    plt.show()

graph_CLT(0, 1)

When testing this out with different mean and standard deviation values, we should still get a similar result. So, let's try a = 20 and b = 50.

Once again, we see that as the sample size grows, the distribution is centered around 0 and resembles the normal distribution.


In [ ]:
graph_CLT(20, 50)